In [1]:
import torch

import numpy as np
import pandas as pd

In [2]:
def accuracy(model, data_x, data_y, pct_close):
    n_items = len(data_y)
    X = torch.Tensor(data_x)  # 2-d torchensor
    Y = torch.Tensor(data_y)  # actual as 1-d torchensor
    oupt = model(X)  # all predicted as 2-d torchensor
    pred = oupt.view(n_items)  # all predicted as 1-d
    n_correct = torch.sum((torch.abs(pred - Y) < torch.abs(pct_close * Y)))
    result = (n_correct.item() * 100.0 / n_items)  # scalar
    return result

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = torch.nn.Linear(79, 178)  # 79-(61-61)-1
        self.hid2 = torch.nn.Linear(178, 178)
        self.hid3 = torch.nn.Linear(178, 178)
        self.oupt = torch.nn.Linear(178, 1)
        torch.nn.init.xavier_uniform_(self.hid1.weight)  # glorot
        torch.nn.init.zeros_(self.hid1.bias)
        torch.nn.init.xavier_uniform_(self.hid2.weight)
        torch.nn.init.zeros_(self.hid2.bias)
        torch.nn.init.xavier_uniform_(self.hid3.weight)
        torch.nn.init.zeros_(self.hid3.bias)
        torch.nn.init.xavier_uniform_(self.oupt.weight)
        torch.nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = torch.relu(self.hid1(x))
        z = torch.relu(self.hid2(z))
        z = torch.relu(self.hid3(z))
        z = self.oupt(z)  # no activation, aka Identity()
        return z

In [4]:
train_fp = '/home/anthony/src/school/cs570/cs570/project3/data/train_norm_num.csv'
test_fp = '/home/anthony/src/school/cs570/cs570/project3/data/test_norm_num.csv'

In [7]:
train_x = np.loadtxt(train_fp, delimiter=",", skiprows=1, usecols=range(1, 80), dtype=np.float32)
train_y = np.loadtxt(train_fp, delimiter=",", skiprows=1, usecols=[80], dtype=np.float32)
test_x_id = np.loadtxt(test_fp, delimiter=",", skiprows=1, usecols=range(0, 1), dtype=np.float32)
test_x = np.loadtxt(test_fp, delimiter=",", skiprows=1, usecols=range(1, 80), dtype=np.float32)

In [8]:
net = Net()

In [9]:
net = net.train()  # set training mode
bat_size = 10

loss_func = torch.nn.MSELoss()  # mean squared error
optimizer = torch.optim.Adam(net.parameters())

n_items = len(train_x)
batches_per_epoch = n_items // bat_size
max_batches = 1000 * batches_per_epoch

for b in range(max_batches):
    curr_bat = np.random.choice(n_items, bat_size, replace=False)
    X = torch.Tensor(train_x[curr_bat])
    Y = torch.Tensor(train_y[curr_bat]).view(bat_size,1)

    optimizer.zero_grad()
    oupt = net(X)
    loss_obj = loss_func(oupt, Y)
    loss_obj.backward()
    optimizer.step()

    if b % (max_batches // 10) == 0 or b == max_batches:
        print("batch = %6d" % b, end="")
        print("  batch loss = %7.4f" % loss_obj.item(), end="")
        net = net.eval()
        acc = accuracy(net, train_x, train_y, 0.15)
        net = net.train()
        print("  accuracy = %0.2f%%" % acc)


batch =      0  batch loss =  0.0416  accuracy = 28.77%
batch =  14600  batch loss =  0.0001  accuracy = 98.84%
batch =  29200  batch loss =  0.0001  accuracy = 98.90%
batch =  43800  batch loss =  0.0000  accuracy = 99.38%
batch =  58400  batch loss =  0.0000  accuracy = 97.95%
batch =  73000  batch loss =  0.0000  accuracy = 99.52%
batch =  87600  batch loss =  0.0000  accuracy = 99.52%
batch = 102200  batch loss =  0.0000  accuracy = 99.59%
batch = 116800  batch loss =  0.0000  accuracy = 99.66%
batch = 131400  batch loss =  0.0000  accuracy = 99.52%


In [10]:
torch.save(net.state_dict(), 'sub2.pth')

In [ ]:
10-10, 61-61 ended with .0002 loss and like 68%

batch =      0  batch loss =  0.1129  accuracy = 4.04%
batch =  14600  batch loss =  0.0012  accuracy = 47.60%
batch =  29200  batch loss =  0.0006  accuracy = 56.16%
batch =  43800  batch loss =  0.0009  accuracy = 59.45%
batch =  58400  batch loss =  0.0007  accuracy = 61.78%
batch =  73000  batch loss =  0.0027  accuracy = 62.33%
batch =  87600  batch loss =  0.0032  accuracy = 63.70%
batch = 102200  batch loss =  0.0010  accuracy = 65.34%
batch = 116800  batch loss =  0.0011  accuracy = 65.82%
batch = 131400  batch loss =  0.0008  accuracy = 66.58%

In [ ]:
16-16

batch =      0  batch loss =  0.4401  accuracy = 4.86%
batch =  14600  batch loss =  0.0017  accuracy = 52.60%
batch =  29200  batch loss =  0.0016  accuracy = 58.63%
batch =  43800  batch loss =  0.0011  accuracy = 58.77%
batch =  58400  batch loss =  0.0011  accuracy = 63.63%
batch =  73000  batch loss =  0.0012  accuracy = 64.25%
batch =  87600  batch loss =  0.0012  accuracy = 65.62%
batch = 102200  batch loss =  0.0008  accuracy = 66.44%
batch = 116800  batch loss =  0.0007  accuracy = 66.78%
batch = 131400  batch loss =  0.0007  accuracy = 66.71%

In [ ]:
30-30

batch =      0  batch loss =  0.5581  accuracy = 1.23%
batch =  14600  batch loss =  0.0004  accuracy = 60.68%
batch =  29200  batch loss =  0.0010  accuracy = 64.38%
batch =  43800  batch loss =  0.0150  accuracy = 66.37%
batch =  58400  batch loss =  0.0013  accuracy = 67.60%
batch =  73000  batch loss =  0.0003  accuracy = 68.56%
batch =  87600  batch loss =  0.0008  accuracy = 69.11%
batch = 102200  batch loss =  0.0012  accuracy = 67.19%
batch = 116800  batch loss =  0.0004  accuracy = 69.45%
batch = 131400  batch loss =  0.0005  accuracy = 70.75%

In [ ]:
178-178-178

batch =      0  batch loss =  0.0188  accuracy = 15.75%
batch =  14600  batch loss =  0.0024  accuracy = 62.47%
batch =  29200  batch loss =  0.0004  accuracy = 66.92%
batch =  43800  batch loss =  0.0084  accuracy = 61.64%
batch =  58400  batch loss =  0.0003  accuracy = 67.81%
batch =  73000  batch loss =  0.0003  accuracy = 71.37%
batch =  87600  batch loss =  0.0003  accuracy = 69.93%
batch = 102200  batch loss =  0.0011  accuracy = 72.60%
batch = 116800  batch loss =  0.0010  accuracy = 71.85%
batch = 131400  batch loss =  0.0004  accuracy = 74.32%

In [ ]:
178-178-178-relu

batch =      0  batch loss =  0.0168  accuracy = 21.58%
batch =  14600  batch loss =  0.0012  accuracy = 68.56%
batch =  29200  batch loss =  0.0006  accuracy = 75.62%
batch =  43800  batch loss =  0.0003  accuracy = 79.59%
batch =  58400  batch loss =  0.0007  accuracy = 83.01%
batch =  73000  batch loss =  0.0003  accuracy = 86.03%
batch =  87600  batch loss =  0.0004  accuracy = 89.18%
batch = 102200  batch loss =  0.0002  accuracy = 91.10%
batch = 116800  batch loss =  0.0002  accuracy = 93.56%
batch = 131400  batch loss =  0.0002  accuracy = 94.52%

In [ ]:
178-178-178-relu-adam

batch =      0  batch loss =  0.0587  accuracy = 0.00%
batch =  14600  batch loss =  0.0003  accuracy = 79.86%
batch =  29200  batch loss =  0.0002  accuracy = 80.82%
batch =  43800  batch loss =  0.0001  accuracy = 81.99%
batch =  58400  batch loss =  0.0006  accuracy = 85.68%
batch =  73000  batch loss =  0.0003  accuracy = 92.19%
batch =  87600  batch loss =  0.0001  accuracy = 91.37%
batch = 102200  batch loss =  0.0001  accuracy = 93.01%
batch = 116800  batch loss =  0.0002  accuracy = 91.30%
batch = 131400  batch loss =  0.0003  accuracy = 93.15%

In [ ]:
sub2

batch =      0  batch loss =  0.0416  accuracy = 28.77%
batch =  14600  batch loss =  0.0001  accuracy = 98.84%
batch =  29200  batch loss =  0.0001  accuracy = 98.90%
batch =  43800  batch loss =  0.0000  accuracy = 99.38%
batch =  58400  batch loss =  0.0000  accuracy = 97.95%
batch =  73000  batch loss =  0.0000  accuracy = 99.52%
batch =  87600  batch loss =  0.0000  accuracy = 99.52%
batch = 102200  batch loss =  0.0000  accuracy = 99.59%
batch = 116800  batch loss =  0.0000  accuracy = 99.66%
batch = 131400  batch loss =  0.0000  accuracy = 99.52%

In [ ]:
net = net.eval()  # set eval mode
# acc = accuracy(net, train_x, train_y, 0.15)
# print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:
pred = net(torch.Tensor(test_x[0]))

In [ ]:
pred

In [ ]:
loaded = Net()
loaded.load_state_dict(torch.load('norm.pth'))
loaded = loaded.eval()
loaded(torch.Tensor(train_x[0]))[0].item()